Pergunta 1:

In [0]:
%sql

-- Pergunta 1 em SQL

-- Corrigindo UF null
WITH UF_Correcao AS (
  SELECT CodigoCidade, MAX(UF) AS UFCorr 
  FROM dlojas
  GROUP BY CodigoCidade
),

dLojas_corrigida AS (
  SELECT StoreID, UFCorr 
  FROM dlojas 
  LEFT JOIN UF_Correcao ON dlojas.CodigoCidade = UF_Correcao.CodigoCidade
),

-- Identificando Classe Social que mais comprou
Classe_top AS (
  SELECT 
    ClasseSocial, 
    SUM(CAST(UnitsSold AS FLOAT) * CAST(REPLACE(ValueSold, ',', '.') AS FLOAT)) AS TOTAL 
  FROM 
    fatovendas
  LEFT JOIN 
    dclientes ON fatovendas.CustomerID = dclientes.CustomerID
  GROUP BY 
    ClasseSocial
  ORDER BY TOTAL DESC
  LIMIT 1
),

-- Filtrando Classe Social que mais comprou na fato
fatoFiltrada AS (
  SELECT *
  FROM
  fatovendas
  LEFT JOIN dclientes on fatovendas.CustomerID = dclientes.CustomerID
  JOIN Classe_top on Classe_top.ClasseSocial = dclientes.ClasseSocial
)

SELECT UFCorr, SUM(CAST(Promo AS FLOAT))
FROM fatoFiltrada FT
LEFT JOIN dLojas_corrigida L ON L.StoreID = FT.StoreID
GROUP BY L.UFCorr

UFCorr,sum(CAST(Promo AS FLOAT))
SP,187.0
RS,174.0
RJ,84.0


In [0]:
## Pergunta 1 em Python

import pandas as pd
# Carregando as tabelas do Spark e convertendo para DataFrame do Pandas
dlojas = spark.table('dlojas').toPandas()
fatovendas = spark.table('fatovendas').toPandas()
dclientes = spark.table('dclientes').toPandas()

# Corrigindo UF nulo
uf_correcao = dlojas.groupby('CodigoCidade')['UF'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
dlojas = dlojas[['StoreID', 'CodigoCidade']].merge(uf_correcao, on='CodigoCidade', how='left')

fatovendas = fatovendas.merge(dclientes, on='CustomerID', how='left')

fatovendas['Promo'] = fatovendas['Promo'].astype(int)
fatovendas['UnitsSold'] = fatovendas['UnitsSold'].astype(float)
fatovendas['ValueSold'] = fatovendas['ValueSold'].str.replace(',', '.').astype(float)
fatovendas['TotalValue'] = fatovendas['UnitsSold'] * fatovendas['ValueSold']

# Calculando classe com maior valor total vendido
classe_top = fatovendas.groupby('ClasseSocial')['TotalValue'].sum().idxmax()

# Filtrando as vendas pela Classe Social que mais comprou
fato_filtrada = fatovendas[fatovendas['ClasseSocial'] == classe_top]

# Agrupando e somando promoções por UF
resultado = fato_filtrada.merge(dlojas[['StoreID', 'UF']], on='StoreID', how='left')
resultado = resultado.groupby('UF')['Promo'].sum()

print(resultado)

UF
RJ     84
RS    174
SP    187
Name: Promo, dtype: int64


Pergunta 2:

In [0]:
%sql

-- Pergunta 2 em SQL

-- Corrigindo UF null
WITH UF_Correcao AS (
  SELECT CodigoCidade, MAX(UF) AS UFCorr 
  FROM dlojas
  GROUP BY CodigoCidade
),

dLojas_corrigida AS (
  SELECT StoreID, UFCorr 
  FROM dlojas 
  LEFT JOIN UF_Correcao ON dlojas.CodigoCidade = UF_Correcao.CodigoCidade
),

-- Clientes que compraram peça azul em apenas 1 UF
Clientes as (
  SELECT CustomerID, COUNT(DISTINCT UFCorr) AS qtd
  FROM fatovendas
  LEFT JOIN dLojas_corrigida on dLojas_corrigida.StoreID = fatovendas.StoreID
  WHERE Cor = 'Azul'
  GROUP BY CustomerID
  HAVING COUNT(DISTINCT UFCorr)=1
),

-- Transações azuis dos clientes selecionados
Azuis as (
  SELECT trim(Tamanho) as Tamanho, sum(FT.UnitsSold) as QTD FROM fatovendas FT
  JOIN Clientes CL on CL.CustomerID = FT.CustomerID
  WHERE Cor = 'Azul'
  GROUP BY trim(Tamanho)
  LIMIT 1
)

SELECT * FROM Azuis

Tamanho,QTD
M,12.0


In [0]:
## Pergunta 2 em Python

import pandas as pd
import numpy as np

# Carregando as tabelas do Spark e convertendo para DataFrame do Pandas
dlojas = spark.table('dlojas').toPandas()
fatovendas = spark.table('fatovendas').toPandas()
dclientes = spark.table('dclientes').toPandas()
fatovendas['UnitsSold'] = fatovendas['UnitsSold'].astype(float)

# Corrigindo UF nulo
uf_correcao = dlojas.groupby('CodigoCidade')['UF'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
dlojas = dlojas[['StoreID', 'CodigoCidade']].merge(uf_correcao, on='CodigoCidade', how='left')

# Clientes que compraram peça azul em apenas 1 UF
fato_lojas_df = pd.merge(fatovendas, dlojas, on='StoreID', how='left')
fato_lojas_df = fato_lojas_df[fato_lojas_df['Cor'] == 'Azul'].groupby('CustomerID')['UF'].nunique()
fato_lojas_df = fato_lojas_df[fato_lojas_df==1]

# Transações azuis dos clientes selecionados
condicao = np.logical_and(fatovendas['Cor'] == 'Azul', fatovendas['CustomerID'].isin(fato_lojas_df.index))
azuis_df = fatovendas[condicao]
azuis_agrupado_df = azuis_df.groupby(azuis_df['Tamanho'].str.strip())['UnitsSold'].sum()
azuis_agrupado_df = azuis_agrupado_df.sort_values(ascending=False).head(1)

# Resultado
print(azuis_agrupado_df)

Tamanho
M    12.0
Name: UnitsSold, dtype: float64


Pergunta 3:

In [0]:
%sql

-- Pergunta 3 em SQL

-- Corrigindo UF null
WITH UF_Correcao AS (
  SELECT CodigoCidade, MAX(UF) AS UFCorr 
  FROM dlojas
  GROUP BY CodigoCidade
),

dLojas_corrigida AS (
  SELECT StoreID, UFCorr 
  FROM dlojas 
  LEFT JOIN UF_Correcao ON dlojas.CodigoCidade = UF_Correcao.CodigoCidade
),

-- Tabela com quantidade de peças vermelhas vendidas por ano
fatoFiltrada AS (
  SELECT `Descrição` as Descricao, substring(`date`, 7, 4) as Ano, SUM(UnitsSold) as Qtd
  FROM fatovendas
  WHERE cor='Vermelho'
  GROUP BY `Descrição`, substring(`date`, 7, 4)
),

-- Item que vendeu mais em 2023 do que 2022
fatoFinal AS (
  SELECT F22.Descricao
  FROM fatoFiltrada f23
  JOIN
  fatoFiltrada F22 on F22.Descricao = F23.Descricao
  WHERE
  F22.Ano = 2022 and F23.Ano = 2023 AND F22.Qtd < F23.Qtd
)

SELECT SUM(cast(UnitsSold as float)) FROM 
fatovendas
JOIN fatoFinal on fatovendas.`Descrição`= fatoFinal.Descricao
LEFT JOIN dLojas_corrigida ON fatovendas.StoreID = dLojas_corrigida.StoreID
WHERE substring(`date`, 7, 4) = '2022' and UFCorr = 'RS'

sum(CAST(UnitsSold AS FLOAT))
230.0


In [0]:
import pandas as pd

# Carregando as tabelas do Spark e convertendo para DataFrame do Pandas
dlojas = spark.table('dlojas').toPandas()
fatovendas = spark.table('fatovendas').toPandas()
dclientes = spark.table('dclientes').toPandas()
fatovendas['UnitsSold'] = fatovendas['UnitsSold'].astype(float)

# Corrigindo UF nulo
uf_correcao = dlojas.groupby('CodigoCidade')['UF'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
dlojas = dlojas[['StoreID', 'CodigoCidade']].merge(uf_correcao, on='CodigoCidade', how='left')

# Tabela com quantidade de peças vermelhas vendidas por ano
fato_filtrada_df = fatovendas[fatovendas['Cor'] == 'Vermelho']
fato_filtrada_df['Ano'] = fato_filtrada_df['Date'].str[6:]
fato_filtrada_df = fato_filtrada_df.groupby(['Descrição', 'Ano'])[['UnitsSold']].sum().reset_index()

# Item que vendeu mais em 2023 do que 2022
fato_2022 = fato_filtrada_df[fato_filtrada_df['Ano'] == '2022']
fato_2023 = fato_filtrada_df[fato_filtrada_df['Ano'] == '2023']

final_df = pd.merge(fato_2022, fato_2023, on='Descrição', how='inner')
final_df = final_df[final_df['UnitsSold_x'] < final_df['UnitsSold_y']][['Descrição']]

# Resultado
merged_df = pd.merge(fatovendas, final_df, on='Descrição', how='inner')
merged_df['Ano'] = merged_df['Date'].str[6:]
merged_df = pd.merge(merged_df, dlojas, on='StoreID', how='left')

resultado = merged_df[(merged_df['Ano'] == '2022') & (merged_df['UF'] == 'RS')]['UnitsSold'].sum()

print(resultado)

230.0


<command-2472483215143544>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fato_filtrada_df['Ano'] = fato_filtrada_df['Date'].str[6:]
